In [267]:
import os
import psycopg2
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
from datetime import datetime

In [268]:
# TMP
os.environ['POSTGRES_HOST'] = "db"

In [269]:
conn = psycopg2.connect(f"dbname='{os.environ['POSTGRES_DB']}' user='{os.environ['POSTGRES_USER']}' host='{os.environ['POSTGRES_HOST']}' password='{os.environ['POSTGRES_PASSWORD']}'")

In [270]:
curs = conn.cursor()
curs.execute("SELECT version()")
print(curs.fetchone())
curs.close()

('PostgreSQL 17.0 (Debian 17.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [271]:
with open('../mt-collector-bot/ships.json', 'r') as file:
    data = json.load(file)

In [272]:
df = pd.DataFrame.from_dict(data, orient='index')

numeric_cols = ['LAT', 'LON', 'SPEED', 'COURSE', 'HEADING', 'ELAPSED', 'LENGTH', 'ROT', 'WIDTH',
                'L_FORE', 'W_LEFT', 'DWT', 'GT_SHIPTYPE']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

int_cols = ['SPEED', 'COURSE', 'HEADING', 'ELAPSED', 'LENGTH', 'ROT', 'WIDTH', 'L_FORE', 'W_LEFT', 'DWT', 'GT_SHIPTYPE']
df[int_cols] = df[int_cols].astype('Int64')

df.replace({np.nan: None}, inplace=True)
df['FLAG'] = df['FLAG'].replace('--', None)
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], unit='s')
df

,LAT,LON,SPEED,COURSE,HEADING,ELAPSED,DESTINATION,FLAG,LENGTH,ROT,...,WIDTH,L_FORE,W_LEFT,DWT,GT_SHIPTYPE,TILE_Z,TIMESTAMP,TYPE_IMG,TYPE_NAME,STATUS_NAME
458357,35.36712,139.8652,0,212,112,2,JP KZU,PA,340,0,...,60,296,25,326964,8,6,2024-10-22 10:43:22.011955023,None,None,None
T0RrME1ERXhPRGswTURFeE9EazBNQT09LUxJT0Y4Rzlya1ZLemVROXViMlBkdnc9PQ==,30.71326,137.9649,80,264,None,772,None,None,None,None,...,None,None,None,None,None,4,2024-10-22 10:30:31.941538095,8,Tanker,Underway using Engine
460530,34.51328,139.5251,7,359,14,2,JP CHB,LR,335,0,...,60,286,35,313990,17,6,2024-10-22 10:43:22.011956930,None,None,None
5304874,35.19753,141.2769,105,107,120,422,JP CHB,JP,339,0,...,60,286,25,311925,17,6,2024-10-22 10:36:22.011957884,None,None,None
T1RNeU9EQTBPVE15T0RBME9UTXlPQT09LXhXRHRxTTR2Q2VaV01lVXBNaHB1Mmc9PQ==,35.03119,141.9140,14,28,None,151,None,None,None,None,...,None,None,None,None,None,4,2024-10-22 10:40:52.941540003,8,Tanker,Underway using Engine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TXpVeE9UUXlNelV4T1RReU16VXhPUT09LVFYOWYxbnk5bG15aUt4UmVNYXBINHc9PQ==,17.51216,157.7756,11,192,None,1420,None,None,None,None,...,None,None,None,None,None,6,2024-10-22 10:19:44.027817011,0,Unspecified Ship,Unknown
TVRjek16YzVNVGN6TXpjNU1UY3pNdz09LTk2K25tMDJKZk43WHJ3R21FbFJvK2c9PQ==,16.20458,150.2494,58,106,None,189,None,None,None,None,...,None,None,None,None,None,6,2024-10-22 10:40:15.027817011,2,Fishing,Default
T0RNMk16azBPRE0yTXprME9ETTJNdz09LXVibU1OUTBDQ0VTTDM2MlloM3FyQkE9PQ==,14.97287,156.1127,63,91,None,713,None,None,None,None,...,None,None,None,None,None,6,2024-10-22 10:31:31.027817965,2,Fishing,Default
TlRNek1UazJOVE16TVRrMk5UTXpNUT09LVVQVTZWSVdWM2VTSkNyVmIzNXorRXc9PQ==,12.78669,154.1663,56,90,None,555,None,None,None,None,...,None,None,None,None,None,6,2024-10-22 10:34:09.027817965,2,Fishing,Default


In [273]:
# Add missing flags

try:
  with conn.cursor() as cursor:
    cursor.execute("SELECT flag FROM flags")
    existing_flags = [row[0] for row in cursor.fetchall()]
    existing_flags
    missing_flags = [{'flag': flag} for flag in df['FLAG'].unique() if flag not in existing_flags and flag is not None]

    insert_query = """
      INSERT INTO flags (flag)
      VALUES (%(flag)s)
    """
    cursor.executemany(insert_query, missing_flags)
    conn.commit()
    print(f"Added {len(missing_flags)} flags ({len(existing_flags)} already existing)")
    cursor.execute("SELECT id, flag FROM flags")
    # Load flags DB
    flags = {}
    for f in cursor.fetchall():
      flags[f[1]] = f[0]
except Exception as e:
  conn.rollback()
  print(f"Error processing: {e}")

Added 1 flags (207 already existing)


In [274]:
# Add missing destinations

try:
  with conn.cursor() as cursor:
    cursor.execute("SELECT name FROM destinations")
    existing_destinations = [row[0] for row in cursor.fetchall()]
    existing_destinations
    missing_destinations = [{'name': dest} for dest in df['DESTINATION'].unique() if dest not in existing_destinations and dest is not None]

    insert_query = """
      INSERT INTO destinations (name)
      VALUES (%(name)s)
    """
    cursor.executemany(insert_query, missing_destinations)
    conn.commit()
    print(f"Added {len(missing_destinations)} destinations ({len(existing_destinations)} already existing)")
except Exception as e:
  conn.rollback()
  print(f"Error processing: {e}")

Added 109 destinations (9663 already existing)


In [277]:
# Register parse event

try:
  with conn.cursor() as cursor:
    insert_query = """
      INSERT INTO parses (start, "end", description)
      VALUES (%(start)s, %(end)s, %(description)s)
      RETURNING id
    """
    d = {
        'start': df['TIMESTAMP'].min(),
        'end': df['TIMESTAMP'].max(),
        'description': 'TEST PARSER WS-MARK',
    }
    cursor.execute(insert_query, d)
    parses_id = cursor.fetchone()[0]
    conn.commit()
except Exception as e:
    conn.rollback()
    print(f"Error processing: {e}")

In [278]:
# Add missing ships and record new ships positions

try:
  for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    with conn.cursor() as cursor:
      cursor.execute("SELECT id FROM ships WHERE ship_id = %s LIMIT 1", (index,))
      ship = cursor.fetchone()
      if not ship:
        insert_query = """
          INSERT INTO ships (ship_id, name, flag_id, width, l_fore, w_left, length)
          VALUES (%(ship_id)s, %(shipname)s, %(flag_id)s, %(width)s, %(l_fore)s, %(w_left)s, %(length)s)
          RETURNING id
        """
        d = {
            'ship_id': row['SHIP_ID'],
            'shipname': row['SHIPNAME'],
            'flag_id': flags[row['FLAG']] if row['FLAG'] is not None else None,
            'width': row['WIDTH'],
            'l_fore': row['L_FORE'],
            'w_left': row['W_LEFT'],
            'length': row['LENGTH']
        }
        cursor.execute(insert_query, d)
        ship_id = cursor.fetchone()[0]
        conn.commit()
      else:
        ship_id = ship[0]
      insert_query = """
          INSERT INTO positions (ship_id, timestamp, location, speed, course, heading, rot, dwt, type, gt_type, parse_id, destination)
          VALUES (%(ship_id)s, %(timestamp)s, %(location)s, %(speed)s, %(course)s, %(heading)s, %(rot)s, %(dwt)s, %(type)s, %(gt_type)s, %(parse_id)s, %(destination)s)
        """
      d = {
          'ship_id': ship_id,
          'timestamp': row['TIMESTAMP'],
          'location': f"({row['LAT']}, {row['LON']})",
          'speed': row['SPEED'],
          'course': row['COURSE'],
          'heading': row['HEADING'],
          'rot': row['ROT'],
          'dwt': row['DWT'],
          'type': row['SHIPTYPE'],
          'gt_type': row['GT_SHIPTYPE'],
          'parse_id': parses_id,
          'destination': row['DESTINATION']
      }
      cursor.execute(insert_query, d)
      conn.commit()

except Exception as e:
    conn.rollback()
    print(f"Error processing: {e}")

100%|██████████| 17901/17901 [02:32<00:00, 117.76it/s]
